In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=4,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=255c6fdfc3e147bfb05f1a456fbffa5c in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=4 image_uid=a5455733a7ec48b1a56ed6814d093c1d in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.91it/s]


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 32.87it/s]


Uploading: numpy-data


In [8]:
dataset_storage_permissions = node_low.python_node.get_service(
    "datasetservice"
).stash.partition.storage_permissions
dataset_id = client_low.datasets[0].id

assert dataset_storage_permissions[dataset_id] == {node_low.python_node.id}

# Data Scientist: make requests

In [9]:
@sy.syft_function()
def square_number(data: float) -> float:
    print("executing sub function")
    return data**2


@sy.syft_function()
def aggregate(data: list[float]) -> float:
    return sum(data) / len(data)


client_low_ds.code.submit(square_number)
client_low_ds.code.submit(aggregate)

SyftSuccess: Syft function 'square_number' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'aggregate' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: User Code Submitted

In [10]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(domain, data) -> float:
    print("executing main function")
    results = []
    for item in data:
        job = domain.launch_job(square_number, data=item)
        results.append(job.result)
    job = domain.launch_job(aggregate, data=results)
    return job.result


# res = compute_mean(data=data_low.mock, blocking=False)
# res.wait().get()

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [11]:
client_low_ds.code

In [12]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [13]:
client_low_ds.code.compute_mean(data=data_low)

SyftError: Execution denied, Please wait for the code to be approved

## Data Owner: Sync to high side

In [14]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [15]:
low_state

syft.service.sync.sync_state.SyncState

In [16]:
assert (
    set(low_state.objects.keys())
    == set(low_state.permissions.keys())
    == set(low_state.storage_permissions.keys())
)

In [17]:
diff_state = compare_states(low_state, high_state)

In [18]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [19]:
diff_state.hierarchies[0].diffs

[ObjectDiff[UserCodeStatusCollection](#9ad04f4d44c5489d86ded8b10be76192),
 ObjectDiff[UserCode](#4b30f52697574c449f80fb5884e85e61),
 ObjectDiff[UserCode](#44ebd9e47fd1439b9574eb7ecf8ff989),
 ObjectDiff[UserCode](#7def01fdce4046189fe36d6cdac5dd3a)]

In [20]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #4b30f52697574c449f80fb5884e85e61:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]
worker_pool_name: default-pool

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #9ad04f4d44c5489d86ded8b10be76192:
    approved: False
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #44ebd9e47fd1439b9574eb7ecf8ff989:
    service_func_name: aggregate
    input_owners: [
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Statu

In [21]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, syft.service.code.user_code.UserCode, syft.service.code.user_code.UserCode, syft.service.request.request.Request, {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [22]:
assert len(resolved_state_high.new_permissions) == 0

new_objs = resolved_state_high.create_objs
assert {o.id for o in new_objs} == {
    sp.uid for sp in resolved_state_high.new_storage_permissions
}

In [23]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 0 items

In [24]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 7 items

In [25]:
request_storage_permissions = node_high.python_node.get_service(
    "requestservice"
).stash.partition.storage_permissions
request_id = client_high.requests[0].id
node_high_id = node_high.python_node.id

assert node_high_id in request_storage_permissions[request_id]

# Data Owner: Run code high-side

In [26]:
data_high = client_high.datasets[0].assets[0]

In [27]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 4c5a84d225ef445e8bc0947b9bad34d8
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [28]:
# wait for the result
job_high.wait().get()

19/03/24 10:22:41 FUNCTION LOG (4c5a84d225ef445e8bc0947b9bad34d8): executing main function
19/03/24 10:22:43 FUNCTION LOG (78b7eeb31261406a88c3ea04a7919213): executing sub function
19/03/24 10:22:44 FUNCTION LOG (ed98ee14dc224746b5425a2774c46155): executing sub function
19/03/24 10:22:44 FUNCTION LOG (6ae30b6ac4a84230a4d434791fec9b23): executing sub function
19/03/24 10:22:45 FUNCTION LOG (dd82da7bd4b34b19b0878cd29e930242): executing sub function
19/03/24 10:22:45 FUNCTION LOG (b020eefca53949d486d5b58973b1512d): executing sub function


291.0

In [29]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [30]:
job_info

syft.service.job.job_stash.JobInfo

In [31]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [32]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: 34f851992a0f49768254043a81c8bc48 as c879d20285a760e0ba5b6417b7326f59aca64f093607e08c2ab4b819fb8df291] 34f851992a0f49768254043a81c8bc48
Job(4c5a84d225ef445e8bc0947b9bad34d8) Setting new result 34f851992a0f49768254043a81c8bc48 -> 34f851992a0f49768254043a81c8bc48


SyftSuccess: Request submitted for updating result.

In [33]:
# Need to refresh Job because we overwrite result
job_high = client_high.code.compute_mean.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [34]:
assert action_store_high.storage_permissions[job_high.result.id.id] == {
    node_high.python_node.id
}

## Data Owner: Sync back to low-side

In [35]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

if isinstance(low_state, sy.SyftError):
    print("low")
    print(low_state.message)

if isinstance(high_state, sy.SyftError):
    print("high")
    print(high_state.message)

REFRESHING a39a1b152d634232b41e2b5d12f45a46
REFRESHING f7c9d19327964e05bb639356e20042d5
REFRESHING f92eacea021445a1af142c7d89d83db9
REFRESHING 3e8a38542ecc4a40885f803f7d8a5992
REFRESHING 513e2e3b6270403bb2be218ac33612c8
REFRESHING 34f851992a0f49768254043a81c8bc48
REFRESHING d611c06e518e4cdd9a82732291675825


In [36]:
aos = [item for item in high_state.objects.values() if isinstance(item, ActionObject)]

for ao in aos:
    print(
        ao.id, ao.syft_action_data_cache, ao.syft_action_data_repr_, ao.syft_action_data
    )
    ao = ao.refresh_object(resolve_nested=False)
    print(ao.syft_action_data_cache, ao.syft_action_data_repr_, ao.syft_action_data)
    print(ao.syft_node_location)
    print(ao)
    print()

a39a1b152d634232b41e2b5d12f45a46 289 289 289
ActionDataEmpty <<class 'numpy.number'>> 289 289
8212e6797fde4c3fba4fc53ab555a886
289

f7c9d19327964e05bb639356e20042d5 256 256 256
ActionDataEmpty <<class 'numpy.number'>> 256 256
8212e6797fde4c3fba4fc53ab555a886
256

f92eacea021445a1af142c7d89d83db9 291.0 291.0 291.0
ActionDataEmpty <<class 'numpy.number'>> 291.0 291.0
8212e6797fde4c3fba4fc53ab555a886
291.0

3e8a38542ecc4a40885f803f7d8a5992 361 361 361
ActionDataEmpty <<class 'numpy.number'>> 361 361
8212e6797fde4c3fba4fc53ab555a886
361

513e2e3b6270403bb2be218ac33612c8 225 225 225
ActionDataEmpty <<class 'numpy.number'>> 225 225
8212e6797fde4c3fba4fc53ab555a886
225

34f851992a0f49768254043a81c8bc48 syft.service.action.action_data_empty.ActionDataLink ```python
class ActionDataLink:
  id: str = 73b8c6fe8f724c40b184d2e6f095afe9

``` syft.service.action.action_data_empty.ActionDataLink
ActionDataEmpty <None> ```python
class ActionDataLink:
  id: str = 73b8c6fe8f724c40b184d2e6f095afe9

``` sy

In [37]:
diff_state_2 = compare_states(low_state, high_state)

In [38]:
resolved_state_low, resolved_state_high = resolve(
    diff_state_2, decision="high", share_private_objects=True
)

LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #4b30f52697574c449f80fb5884e85e61:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #9ad04f4d44c5489d86ded8b10be76192:
    status_dict: (<UserCodeStatus.PENDING: 'pending'>, '')

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #44ebd9e47fd1439b9574eb7ecf8ff989:
    service_func_name: aggregate
    input_owners: [
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #7def01fdce4046189fe36d6cdac5dd3a:
    service_func_name: square_number
    input_owners: [
    ]


In [39]:
print(resolved_state_low)
print()
print(resolved_state_high)

ResolvedSyncState(
  create_objs=[syft.service.output.output_service.ExecutionOutput, Pointer:
syft.service.action.action_data_empty.ActionDataLink, syft.service.job.job_stash.Job, syft.service.job.job_stash.Job, Pointer:
256, syft.service.log.log.SyftLog, syft.service.job.job_stash.Job, Pointer:
225, syft.service.log.log.SyftLog, syft.service.job.job_stash.Job, syft.service.log.log.SyftLog, Pointer:
324, syft.service.job.job_stash.Job, Pointer:
289, syft.service.log.log.SyftLog, syft.service.job.job_stash.Job, Pointer:
291.0, syft.service.log.log.SyftLog, syft.service.log.log.SyftLog, syft.service.job.job_stash.Job, syft.service.log.log.SyftLog, Pointer:
361],
  update_objs=[{NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}, syft.service.request.request.Request],
  delete_objs=[]
  new_permissions=[[READ: 34f851992a0f49768254043a81c8bc48 as c879d20285a760e0ba5b6417b7326f59aca64f093607e08c2ab4b819fb8df291], [READ: 4c5a84d225ef445e8bc0947

In [40]:
aos = [
    item for item in resolved_state_low.create_objs if isinstance(item, ActionObject)
]
for ao in aos:
    print(ao.id, ao.syft_action_data_cache)

34f851992a0f49768254043a81c8bc48 syft.service.action.action_data_empty.ActionDataLink
f7c9d19327964e05bb639356e20042d5 256
513e2e3b6270403bb2be218ac33612c8 225
d611c06e518e4cdd9a82732291675825 324
a39a1b152d634232b41e2b5d12f45a46 289
f92eacea021445a1af142c7d89d83db9 291.0
3e8a38542ecc4a40885f803f7d8a5992 361


In [41]:
res = client_low.apply_state(resolved_state_low)
res

SyftSuccess: Synced 24 items

In [42]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 0 items

In [43]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [44]:
result_storage_permissions = node_low.python_node.get_service(
    "actionservice"
).store.storage_permissions[job_high.result.id.id]
assert len(result_storage_permissions) == 1

log_storage_permissions = node_low.python_node.get_service(
    "logservice"
).stash.partition.storage_permissions[job_high.log_id]
assert len(log_storage_permissions) == 1

# Run code low

## Run

In [45]:
client_low_ds._fetch_api(client_low.credentials)

In [46]:
client_low_ds.code.compute_mean.jobs[0].result.get()

291.0

In [47]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
291.0

In [48]:
job_high_result_resolved = job_high.result.refresh_object(resolve_nested=True)

In [60]:
res_low.id == job_high_result_resolved.id

True

In [57]:
client_low.code[-1]

```python
class UserCode
    id: UID = 4b30f52697574c449f80fb5884e85e61
    service_func_name: str = compute_mean
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: approved']
    
    code:

@sy.syft_function_single_use(data=data_low)
def compute_mean(domain, data) -> float:
    print("executing main function")
    results = []
    for item in data:
        job = domain.launch_job(square_number, data=item)
        results.append(job.result)
    job = domain.launch_job(aggregate, data=results)
    return job.result



  Nested Requests:
  class UserCode
      id: UID = 7def01fdce4046189fe36d6cdac5dd3a
      service_func_name: str = square_number
      shareholders: list = []
      status: list = ['Node: test_l, Status: pending']
      
      code:
  
  @sy.syft_function()
  def square_number(data: float) -> float:
      print("executing sub function")
      return data ** 2
  
  class UserCode
      id: UID = 44ebd9e47fd1439b9574eb7ecf8ff989
      service_func_name: str = aggregate
      shareholders: list = []
      status: list = ['Node: test_l, Status: pending']
      
      code:
  
  @sy.syft_function()
  def aggregate(data: list[float]) -> float:
      return sum(data) / len(data)
  
```

In [63]:
code = client_low_ds.code[-1]

assert res_low.get() == np.square(private_high).mean()
assert res_low.id.id == job_high_result_resolved.id.id
assert (
    job_high_result_resolved.syft_blob_storage_entry_id
    == res_low.syft_blob_storage_entry_id
)

In [71]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [72]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

AssertionError: 

In [73]:
private_high.mean()

17.0

In [74]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = d76d16a7bf3341deb0caccd402bb4fd5
    status: completed
    has_parent: False
    result: ActionDataEmpty <None>
    logs:

0 Log 8ad86aa308364ea2a98ce8cb4341093f not available
JOB COMPLETED
    
```

In [75]:
job_low.wait().get()

17.0

In [76]:
job_low.logs()

Log 8ad86aa308364ea2a98ce8cb4341093f not available


In [77]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)